In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd

In [38]:
from ecephys.sglx_utils import load_timeseries
from ecephys.data import channel_groups, paths
from ecephys.plot import timeseries_explorer, colormesh_timeseries_explorer
from ecephys.signal.utils import mean_subtract, median_subtract

In [22]:
condition = "SR"
subject = "Doppio"

# chans = channel_groups.CA1[subject]
# chans = chans[0:11:2]
chans = channel_groups.full[subject]
# chans = np.delete(chans, chans == 191)
bin_path = Path(paths.lfp_bin[condition][subject])
(times, sig, fs) = load_timeseries(bin_path, chans, start_time=(645), end_time=(645 + 1.0))

nChan: 385, nFileSamp: 18000019


In [43]:
sig = mean_subtract(sig)

## 1D kCSD on whole probe

In [44]:
from kcsd import KCSD1D

n_chans = len(chans)
intersite_distance = 0.020
ele_pos = np.linspace(0., (n_chans - 1) * intersite_distance, n_chans).reshape(n_chans, 1)
pots = sig.T

In [45]:
k = KCSD1D(ele_pos, pots, gdx=0.02)
est_csd = k.values('CSD')
est_csd.shape

(384, 2501)

In [46]:
import matplotlib.cm as cm

_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, k.estm_x, est_csd, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")
hi = 1.6
lo = 1.25
ax.axhline(hi, alpha=0.5, color='k', linestyle="--")
ax.axhline(lo, alpha=0.5, color='k', linestyle="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
chans[np.logical_and(ele_pos >= lo, ele_pos <= hi).flatten()]

array([127, 128, 131, 132, 135, 136, 139, 140, 143, 144, 147, 148, 151,
       152, 155, 156, 159, 160])

In [48]:
np.where(np.logical_and(ele_pos >= lo, ele_pos <= hi).flatten())

(array([63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
        80]),)

In [49]:
chans[190:291]

array([380, 383,   1,   2,   5,   6,   9,  10,  13,  14,  17,  18,  21,
        22,  25,  26,  29,  30,  33,  34,  37,  38,  41,  42,  45,  46,
        49,  50,  53,  54,  57,  58,  61,  62,  65,  66,  69,  70,  73,
        74,  77,  78,  81,  82,  85,  86,  89,  90,  93,  94,  97,  98,
       101, 102, 105, 106, 109, 110, 113, 114, 117, 118, 121, 122, 125,
       126, 129, 130, 133, 134, 137, 138, 141, 142, 145, 146, 149, 150,
       153, 154, 157, 158, 161, 162, 165, 166, 169, 170, 173, 174, 177,
       178, 181, 182, 185, 186, 189, 190, 193, 194, 197])

## 1D kCSD on hippocampal channels

In [50]:
hpc_chans = chans[190:291]
hpc_sig = sig[:, 190:291]

In [51]:
from kcsd import KCSD1D

n_hpc_chans = len(hpc_chans)
intersite_distance = 0.020
ele_pos = np.linspace(0., (n_hpc_chans - 1) * intersite_distance, n_hpc_chans).reshape(n_hpc_chans, 1)
pots = hpc_sig.T

In [52]:
k = KCSD1D(ele_pos, pots, gdx=0.02)
est_csd = k.values('CSD')

In [53]:
import matplotlib.cm as cm

_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, k.estm_x, est_csd, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")
#ax.axhline(5.8, alpha=0.5, color='k', linestyle="--")
#ax.axhline(3.8, alpha=0.5, color='k', linestyle="--")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth (mm)')

In [56]:
timeseries_explorer(times, est_csd.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [57]:
k.L_curve()

No lambda given, using defaults
min lambda 1e-12
max lambda 0.0126
min lambda 1e-12
max lambda 0.0126
l-curve (all lambda):  0.23
Best lambda and R =  3.6037656676605034e-06 ,  0.23


In [58]:
est_csd_val = k.values('CSD')

In [59]:
import matplotlib.cm as cm

_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, k.estm_x, est_csd_val, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth (mm)')

In [60]:
timeseries_explorer(times, est_csd_val.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [64]:
# DATA SHOULD BE MEAN/MEDIAN SUBTRACTED !!!!
# From AllenSDK
def compute_csd(trial_mean_lfp, spacing):
    '''Compute current source density for real or virtual channels from
    a neuropixels probe.
    Compute a second spatial derivative along the probe length
    as a 1D approximation of the Laplacian, after Pitts (1952).
    Parameters
    ----------
    trial_mean_lfp: numpy.ndarray
        LFP traces surrounding presentation of a common stimulus that
        have been averaged over trials. Dimensions are channels X time samples.
    spacing : float
        Distance between channels, in millimeters. This spacing may be
        physical distances between channels or a virtual distance if channels
        have been interpolated to new virtual positions.
    Returns
    -------
    Tuple[csd, csd_channels]:
        csd : numpy.ndarray
            Current source density. Dimensions are channels X time samples.
        csd_channels: numpy.ndarray
            Array of channel indices for CSD.
    '''

    # Need to pad lfp channels for Laplacian approx.
    padded_lfp = np.pad(trial_mean_lfp,
                        pad_width=((1, 1), (0, 0)),
                        mode='edge')

    csd = (1 / (spacing ** 2)) * (padded_lfp[2:, :]
                                  - (2 * padded_lfp[1:-1, :])
                                  + padded_lfp[:-2, :])
    
    csd_channels = np.arange(0, trial_mean_lfp.shape[0])
    
    return (csd, csd_channels)

In [78]:
est_csd_aibs, csd_channels_aibs = compute_csd(pots[1::4], intersite_distance * 4)

In [79]:
_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, csd_channels_aibs, -est_csd_aibs, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth (mm)')

In [80]:
timeseries_explorer(times, -est_csd_aibs.T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

## 2D kCSD on hippocampal channels

In [265]:
def make_2D_plot(xx, yy, zz, title='CSD'):
    fig = plt.figure(figsize=(5, 15))
    ax = plt.subplot(111)
    ax.set_aspect('equal')
    t_max = np.max(np.abs(zz))
    levels = np.linspace(-1 * t_max, t_max, 32)
    im = ax.contourf(xx, yy, zz, levels=levels)
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_title(title)
    ticks = np.linspace(-1 * t_max, t_max, 3, endpoint=True)
    return ax

In [236]:
from ecephys.graham.channel_coords import CheckPat_x, CheckPat_y
(hpc_x, hpc_y) = (CheckPat_x[hpc_chans], CheckPat_y[hpc_chans])

In [ ]:
from ecephys.plot import plot_channel_coords
plot_channel_coords(hpc_chans, hpc_x, hpc_y)

In [249]:
hpc_ele_pos = np.vstack([hpc_x, hpc_y]).T
hpc_pots = hpc_sig.T

In [393]:
from kcsd import KCSD2D

vertical_site_spacing = 20
horizontal_site_spacing = 16
k = KCSD2D(hpc_ele_pos, hpc_pots, gdx=horizontal_site_spacing, gdy=vertical_site_spacing)
est_csd_hpc_2D = k.values('CSD')
k.L_curve()
est_csd_hpc_2D_val = k.values('CSD')


No lambda given, using defaults
min lambda 1e-06
max lambda 0.0001
min lambda 1e-06
max lambda 0.0001
l-curve (all lambda):  0.23
Best lambda and R =  8.857135378437576e-07 ,  0.23


In [394]:
ax = make_2D_plot(k.estm_x, k.estm_y, est_csd_hpc_2D_val[:, :, (np.abs(times - 645.4621)).argmin()])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [395]:
est_csd_flat = np.sum(est_csd_hpc_2D_val, axis=0)

In [413]:
_, ax = plt.subplots(figsize=(20, 10))
ax.pcolormesh(times, k.estm_y[0], est_csd_flat, shading="gouraud")
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Depth (mm)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'KCSD1D' object has no attribute 'estm_y'